<a href="https://colab.research.google.com/github/subhamkapoor360/MLHelpers/blob/master/Mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals


import tensorflow 
import numpy as np
import warnings


from tensorflow.keras.datasets.mnist import load_data
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.preprocessing.image import  array_to_img

warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [0]:
from tensorflow.keras.datasets import mnist


def load_mnist_data():
	'''
	Usage:  X_train, y_train, X_test,y_test = load_fashion_mnist_data()
	'''
	digits = mnist.load_data()
	(X_train, y_train),(X_test, y_test) = digits
	X_train = X_train.astype('float32')
	X_train, X_test = X_train/255.0,X_test/255.0
	return X_train, y_train, X_test,y_test

In [0]:
batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28


In [0]:
X_train, y_train, X_test,y_test = load_mnist_data()
if K.image_data_format() == 'channels_first':
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)


In [0]:
y_train = tensorflow.keras.utils.to_categorical(y_train, num_classes)
y_test = tensorflow.keras.utils.to_categorical(y_test, num_classes)

In [0]:
import tensorflow
from tensorflow.keras.layers import (Dense,Dropout,Conv2D,MaxPooling2D,Flatten)
from tensorflow.keras.activations import (relu,softmax)
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
class ImageClassification(tensorflow.keras.Model):

  def __init__(self,input_shape,n_classes):
    super(ImageClassification, self).__init__()
    self.conv2d_1 = Conv2D(28, kernel_size=(5, 5),
                 activation=relu,
                 input_shape=input_shape)
    self.conv2d_2 = Conv2D(28, (3, 3), activation=relu)
    self.conv2d_3 = Conv2D(64, kernel_size=(5, 5),
                 activation=relu)
    self.conv2d_4 = Conv2D(64, (3, 3), activation=relu)
    self.maxpoolings2d_1 = MaxPooling2D(pool_size=(2, 2))
    self.maxpoolings2d_2 = MaxPooling2D(pool_size=(2, 2))
    
    self.dropout = Dropout(.5)
    self.flatten = Flatten()
    self.dense_1 = Dense(n_classes, activation='softmax',)
    
    

  def call(self, inputs, training=False):
    x = self.conv2d_1(inputs)
    x = self.conv2d_2(x)
    x = self.maxpoolings2d_1(x)
    x = self.dropout(x)
    x = self.conv2d_3(x)
    x = self.conv2d_4(x)
    x = self.maxpoolings2d_2(x)
    x = self.dropout(x)
    x = self.flatten(x)
    return self.dense_1(x)

In [0]:
img_class = ImageClassification((28,28),10)

In [8]:
img_class.compile(optimizer=Adam(.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy',])

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
filepath="Mnist"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max',save_weights_only=True)
earlystopper = EarlyStopping(patience=4, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.1,
                              patience=2, min_lr=0.00000001, verbose=1)
callbacks_list = [checkpoint,earlystopper,reduce_lr]

In [10]:
history = img_class.fit(X_train,y_train,batch_size = 32,epochs=50,validation_data=(X_test, y_test),callbacks=callbacks_list)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Train on 60000 samples, validate on 10000 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/50
59904/60000 [============================>.] - ETA: 0s - loss: 0.7981 - acc: 0.7359
Epoch 00001: val_acc improved from -inf to 0.94910, saving model to Mnist

Consider using a TensorFlow optimizer from `tf.train`.
Instructions for updating:
Use tf.train.CheckpointManager to manage checkpoints rather than manually editing the Checkpoint proto.
60000/60000 [==============================] - 12s 203us/sample - loss: 0.7972 - acc: 0.7363 - val_loss: 0.1802 - val_acc: 0.9491
Epoch 2/50
59840/60000 [============================>.] - ETA: 0s - loss: 0.2799 - acc: 0.9124
Epoch 00002: val_acc improved from 0.94910 to 0.96700, saving model to Mnist

Consider using a TensorFlow optimizer from `tf.train`.
60000/60000 [==============================] - 9s 155us/sample - loss: 0.

In [0]:
image_predictions = img_class.predict(X_test)



In [12]:
img_class.evaluate(X_test,y_test)

10000/10000 [==============================] - 1s 54us/sample - loss: 0.0209 - acc: 0.9939


[0.02085083363817248, 0.9939]

In [0]:
test_index = 101

In [14]:
test_image = X_test[test_index]
array_to_img(test_image)

In [0]:
prediction = image_predictions[test_index].argmax()

In [16]:
prediction

0